# Computer Vision CIFAR-10

En este notebook continuaremos profundizando en las funcionalidades de Keras y Tensorflow, para entrenar Redes Neuronales Convolucionales y resolver problemas de clasificación más complejos. En particualr vamos a resolver el problema de clasificación de imágenes en el dataset CIFAR-10.

# Tabla de Contenidos

Este notebook contiene 5 partes. Vamos a repasar algunas funcionalidades básicas de TensorFlow y Keras entrenando un modelo para reconocer dígitos escritos a mano.

1. Parte I, Preparación: cargar dataset CIFAR-10.
2. Parte II, Entendimiento: visualizar datos y obtener algunas estadísticas.
3. Parte III, Entrenamiento: definición y entrenamiento de una Red Convolucional.
4. Parte IV, Evaluación: evaluar el modelo y predecir algunos casos.
4. Parte V, Experimentación

# Parte I: Preparación

Primero, vamos a descargar el dataset CIFAR10. Esto puede tomar algunos minutos para descargar los datos por primer vez, pero luego de que los archivos son descargados y cacheados en disco la carga debería ser más rápida.

Al igual que en el notebook anterior y por simplicidad, vamos a trabajar con el módulo ```tf.keras.datasets``` que ya provee utilidades para cargar y trabajar con este dataset entre otros.

In [ ]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def load_cifar10(num_training=48000, num_validation=2000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    
    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()

Opcionalmente puedes **usasr GPU seteando la siguiente flag en True** en la siguiente celda.

## Usuarios Colab

Si estas usando Colab, seguramente necesites manualmente cambiar a un entorno GPU. La forma de hacer esto es clickeando la opcion `Runtime -> Change runtime type` y seleccionar la opción `GPU` dentro de `Hardware Accelerator`. Notar que debes correr de nuevo todas las celdas de arriba ya que al cambiar de entorno de ejecución el kernel del notebook se reinicia perdiendo el estado actual.

In [ ]:
# Set up some global variables
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

# Parte II: Entendimiento de los datos

Recordemos algunas de las preguntas que nos solemos hacer antes de empezara a trabajar con un set de datos de entrenamiento: 

* ¿Cuántas imágenes tenemos para Train y cuántas para Test?
* ¿Cuántas imágenes tenemos por cada clase, está balanceado?
* ¿Qué resolución tiene cada imagen? ¿Son todas iguales o hay que aplicar algún re-size?
* ¿Cuantos canales tiene la imágen (blanco y negro o a color)?

Además, es una buena práctica visualizar algunas imágenes por cada clase, para comprender bien como son las imágenes con las que estaremos trabajando.

Empecemos entonces a contestar estas preguntas!!

**¿Cuántas imágenes tenemos?**

Veamos algunas estadísticas generales del dataset.

In [ ]:
# Print some basic stats about the dataset
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

**¿Cuántas imágenes tenemos por cada clase?** Complete la celda a continuación par verificar si los datos de entrenamiento (y_train) se encuentran balanceados.

In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_train y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

**¿Que sucede con el dataset de test?** Complete la celda a continuación para verificar si los datos de prueba (y_test) se encuentran balanceados.

In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_test y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

**¿Qué resolución tiene cada imagen?**

El dataset de entrenamiento se compone de **50.000** imágenes de una misma resolución para facilitar el trabajo de entrenamiento. Para corroborar esto se puede revisar las dimensiones de la matrix que contiene los datos de entrenamiento, es decir, las dimensiones de `X_train`. 

_HINT: Recuerde que toda matrix de `numpy` cuenta con el atributo `shape` que describe sus dimensiones._

In [ ]:
############################################################################
# TODO: Revisar la resolución de una imagen de entrenamiento               #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

**Entendiendo las imágenes**

A continuación vamos a ver de que se tratan las imágenes de CIFAR-10 y para ello vamos a utilizar las misma estrategia que con MNIST: matplotlib. Notese además, a diferencia de MNIST, que en CIFAR-10 las imágenes son a color por lo que tenemos 3 canales extra en las imágenes: RGB.

Por otro lado, en MNIST las imágenes se correspondian a los números 0,1, 2, ... , 9 y sus etiquetas coincidían con el dígito en cuestión. En este caso, las etiquetas son: "airplane", "automobile", ..., "truck". Sin embargo, por simplicidad también se codifican con números del 0 al 9 (en este caso porque tenemos 10 clases). Por ello, será necesario definirnos un diccionario para mapear de los respectivos números en las etiquetas a su clase, para facilitar la interpretación de los resultados.

In [ ]:
# Maps label number to class name

idx_to_label = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

In [ ]:
def display_cifar10_images(X, y, num=25, num_row=5, num_col=5):
    """
    Display on a grid using matplotlib train images and their corresponding
    label.
    """
    
    # Get images to display
    images = X[:num]
    labels = y[:num]

    # plot images
    fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
    for i in range(num):
        ax = axes[i//num_col, i%num_col]
        ax.imshow(images[i])
        ax.set_title('Label: {}'.format(idx_to_label[labels[i][0]]))
        
    plt.tight_layout()
    plt.show()

display_cifar10_images(X_train, y_train)

Vamos a ver una forma diferente de normalizar las fotos. Mientras que en el notebook anterior normalizabamos cada pixel de una foto teniendo en cuenta la media y desviación estandard en el dataset, en este caso simplemente vamos a llevar el valor de un pixel al intervalo [0 ... 1]. Esto es porque las Redes Neuronales realizan las operaciones de forma más eficiente con números en este rango.

In [ ]:
def scale_images(X_train, X_test):
    """
    This function scales each image to range 0-1 for simplicity for
    neural networks.
    """
    
    ############################################################################
    # TODO: Revisar la resolución de una imagen de entrenamiento               #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


    # convert from integers to floats
    train_norm = X_train.astype('float32')
    test_norm = X_test.astype('float32')
    
    # normalize to range 0-1
    train_norm = pass / 255  # TODO: Normalize each pixel in range 0-255
    test_norm = pass / 255  # TODO: Normalize each pixel in range 0-255 
    
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################

    return train_norm, test_norm

X_train, X_test = scale_images(X_train, X_test)

# Parte III: Entrenamiento

Ahora que tenemos un mejor entendimiento del problema y las imágenes con las que contamos, pasemos a definir y entrenar el modelo que clasifique las imágenes.

Para esto vamos a utilizar la clase `tf.keras.Sequential` disponible en la libreria Keras, que permite definir una red neuronal con un número arbitrario de `layers`. Por más información acerca de esta clase leer [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) la documentación.

Para entrar en calentamiento lo que vamos a hacer, es definir una sencilla Red Neuronal utilizando convoluciones y finalmente una capa densamente conectada. 

Resumen de la arquitectura:
* Input Layer: Imágenes de 32x32 que como son el input de una convolución no las vamos a aplanar como vector aun. 
* Conv2D: Capa de convolución con 16 filtros de 3x3 y Relu. Leer más en [tf.keras.layers.Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/).
* Conv2D: Capa de convolución de 32 filtros de 3x3.
* Flatten: Pasamos a vector. Leer más en [tf.keras.layers.Flatten](https://keras.io/api/layers/reshaping_layers/flatten/).
* Hidden Layer: Capa oculta de 64 neuronas con Relu. Leer más sobre [tf.keras.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense).
* Output Layer: Nuevamente usamos `tf.keras.layers.Dense` pero con largo 10 (por 10 diferentes clases).

In [ ]:
# Define model architecture using keras.Sequential API
model = tf.keras.Sequential([
    tf.keras.Input(shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation="relu"),  # 32 blocks of CONV2D
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),  # 64 blocks of CONV2D
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)  # Dense of num_classes length
])

Compilación:

In [ ]:
# Compile model
model.compile(optimizer='SGD',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Entrenamiento

In [ ]:
# Train model
model.fit(X_train, y_train, epochs=10)

# Parte IV: Evaluación

A diferencia del MNIST, este es un problema más complejo de resolver por lo que no es de esperar lograr resultados muy buenos con la arquitectura simple definida arriba. De hecho, entrenando esa misma arquitectura, se debería alcanzar una `loss ~ 1.065` y una `accuracy` en el entorno de `0.629` sobre los datos de Test.

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

### Test con imagen arbitraria

Veamos que tan bien nos va con una imagen cualquiera del dataset de Test.

In [ ]:
# Change num_sample to test with a different image
num_sample = 12
image = X_test[num_sample]
label = y_test[num_sample]

plt.imshow(image)
plt.show()

In [ ]:
predictions = model.predict(np.expand_dims(image, axis=0)) 
prediction = np.argmax(predictions[0])  # prediction is an array of 10 probabilities, the highest is the predicted class

print(f"Prediction: {idx_to_label[prediction]} and expected is: {idx_to_label[label[0]]}")

# Parte IV: Experimentación

Ahora que tenemos claro como definir Redes Neuronales Convolucionales y su poder, te toca el turno a ti de jugar con diferentes opciones de arquitecturas para alcanzar una mejor accuracy sobre los datos de Test.

A modo de guia te sugerimos algunas de las siguientes variantes para probar:

* Agregar layers de convolución: Agregar más capas Conv2D y con mayor profundidad (32, 64, 128).
* Probar intercalar layers de convolución con Max Pooling.  Leer más en [tf.keras.layers.MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/).
* Probar varias layers densas al final para incrementar capacidad del modelo.
* Optimizadores: En el modelo original se utiliza SGD (Stochastic gradient descent), probar con otros optimizadores como [RMSprop o Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers).
* Dropout: 

* Dropout: Para evitar sobreajuste. Leer más en [tf.keras.layers.Dropout](https://keras.io/api/layers/regularization_layers/dropout/).


In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_test y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = tf.keras.Sequential([
    ...  # TODO: Edit this
])

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

In [ ]:
model.summary()

In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_test y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model.compile(optimizer=pass,  # TODO: Select an optimizer
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_test y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model.fit(X_train, y_train, epochs=pass)  # TODO: select num of epochs

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

Un método usualmente útil para diagnosticar si el entrenamiento de un modelo es correcto, es graficar la evolución de alguna de las métricas de evaluación del modelo como la `accuracy` o la `loss` a través de las `epochs`. Además comparar la misma métrica en el dataset de Train y Validation.

Veamos a continuación como podemos hacer esto utilizando `matplotlib` y los valores que el propio método `fit` de TensorFlow nos da.

1. Primero tenemos que cambiar la linea de entrenamiento por lo siguiente, para guardar el histórico de valores de las métricas a evaluar.

```
history = model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val, y_val))
```

2. Luego, utilizamos la información en history para construír gráficos.

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val, y_val))

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    
summarize_diagnostics(history)